In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import PIL
import cv2

from keras import layers
from keras.models import Sequential, Model

from sklearn.metrics import classification_report

from time import time

In [33]:
batch_size = 32
img_height = 224
img_width = 224

In [72]:
from google.colab import drive

drive.mount('/content/drive')

#!ls "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
from sklearn.metrics import classification_report


def evaluate_f1(y_true, y_pred):
    """
    Метод для расчёта avg f-1 score по датасету
    :param y_pred - предсказанные моделью метки классов
    :param y_true - реальные метки классов
    :return значение f-1 score по всем классам цветков
    """
    return classification_report(y_true, y_pred,
                                 target_names=['buttercup', 'coreopsis', 'daffodil', 'dandelion', 'sunflower'],
                                 output_dict=True)['macro avg']['f1-score']

In [76]:
!unzip -u  '/content/drive/MyDrive/validation.zip' -d "/data_lab"

Archive:  /content/drive/MyDrive/validation.zip
   creating: /data_lab/validation/
   creating: /data_lab/validation/buttercup/
  inflating: /data_lab/validation/.DS_Store  
   creating: /data_lab/validation/coreopsis/
   creating: /data_lab/validation/dandelion/
   creating: /data_lab/validation/sunflower/
   creating: /data_lab/validation/daffodil/
  inflating: /data_lab/validation/buttercup/.DS_Store  
  inflating: /data_lab/validation/buttercup/val_buttercup_9.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_8.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_3.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_2.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_1.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_5.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_4.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_6.jpg  
  inflating: /data_lab/validation/buttercup/val_buttercup_7.jpg

In [53]:
!unzip -u  '/content/drive/MyDrive/flowers.zip' -d "/folder"

Archive:  /content/drive/MyDrive/flowers.zip
  inflating: /folder/buttercup/004b865b1e.jpg  
  inflating: /folder/buttercup/01195ccc5f.jpg  
  inflating: /folder/buttercup/01d4312039.jpg  
  inflating: /folder/buttercup/01f8136423.jpg  
  inflating: /folder/buttercup/01R0R2MUP72I.jpg  
  inflating: /folder/buttercup/0277cad84f.jpg  
  inflating: /folder/buttercup/02800fa9be.jpg  
  inflating: /folder/buttercup/036904147c.jpg  
  inflating: /folder/buttercup/03b00535a1.jpg  
  inflating: /folder/buttercup/0460678026.jpg  
  inflating: /folder/buttercup/0509028174.jpg  
  inflating: /folder/buttercup/05673c51c6.jpg  
  inflating: /folder/buttercup/056ea8d6df.jpg  
  inflating: /folder/buttercup/06ab315651.jpg  
  inflating: /folder/buttercup/075d6a2bed.jpg  
  inflating: /folder/buttercup/096cb92fd6.jpg  
  inflating: /folder/buttercup/0a6fb46b66.jpg  
  inflating: /folder/buttercup/0b111b0fc4.jpg  
  inflating: /folder/buttercup/0b8ea22296.jpg  
  inflating: /folder/buttercup/0d1843466a

In [54]:
data_dir = '/folder'

In [55]:
def check_images( s_dir, ext_list):
    s_list= os.listdir(s_dir)
    for klass in s_list:
        klass_path=os.path.join (s_dir, klass)
        if os.path.isdir(klass_path):
            file_list = os.listdir(klass_path)
            for f in file_list:
                f_path = os.path.join (klass_path,f)
                index = f.rfind('.')
                ext = f[index+1:].lower()
                if ext not in ext_list:
                    print('file ', f_path, ' has an invalid extension ', ext)
                    path = f_path.replace(ext,"jpeg")
                    if ext =='webp':
                        img = cv2.imread(f_path)
                        retval, buf = cv2.imencode(".webp", img, [cv2.IMWRITE_WEBP_QUALITY, 100])
                        img = cv2.imdecode(buf, 1)
                        cv2.imwrite(path, img)
                    elif ext == 'jpg':
                        img = PIL.Image.open(f_path)
                        img.save(path)
                    os.remove(f_path)


good_exts=['png', 'jpeg', 'gif', 'bmp'] # list of acceptable extensions
check_images(data_dir, good_exts)

file  /folder/daffodil/46033303295_78cf8d651f_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/4449037222_52aaed3f0b_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/5627625513_78e9bf654c_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/2438141689_227c26b38c_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/5672440378_2a313af58e_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/3352540123_981220e8ab_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/7126340195_404d41f92c_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/487258863_49ff70165f_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/TXHIMUJGNO4F.jpg  has an invalid extension  jpg
file  /folder/daffodil/4463909292_07f9740077_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/3322636879_23f66b1e77_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/348835365_4256ce0bc7_c.jpg  has an invalid extension  jpg
file  /folder/daffodil/ZH443

In [78]:
data_dir = '/data_lab/validation'
check_images(data_dir, good_exts)

file  /data_lab/validation/daffodil/val_daffodil_2.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_6.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_5.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_10.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_9.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_8.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_4.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_7.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_3.jpg  has an invalid extension  jpg
file  /data_lab/validation/daffodil/val_daffodil_1.jpg  has an invalid extension  jpg
file  /data_lab/validation/sunflower/val_sunflower_8.jpg  has an invalid extension  jpg
file  /data_lab/validation/sunflower/val_sunflower_

In [84]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.8,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 54 files belonging to 5 classes.
Using 11 files for training.


In [85]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 54 files belonging to 5 classes.
Using 10 files for validation.


In [86]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 54 files belonging to 5 classes.


In [58]:
class_names = train_ds.class_names
print(class_names)

num_classes = len(class_names)

['buttercup', 'coreopsis', 'daffodil', 'dandelion', 'sunflower']


In [59]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same'),
  layers.BatchNormalization(axis=3),
  layers.Activation('relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same'),
  layers.BatchNormalization(axis=3),
  layers.Activation('relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same'),
  layers.BatchNormalization(axis=3),
  layers.Activation('relu'),
  layers.AveragePooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation="softmax")
])

In [ ]:
AlexNet = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(96, 11, 4, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(3, 2),
    layers.Conv2D(256, 5, 1, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(3, 2),
    layers.Conv2D(384, 3, 1, activation='relu'),
    layers.Conv2D(384, 3, 1, activation='relu'),
    layers.Conv2D(256, 3, 1, activation='relu'),
    layers.MaxPooling2D(3, 2),
    layers.Flatten(),
    layers.Dense(150, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(70, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='sigmoid'),
])

In [ ]:
VGG = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(128, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.MaxPooling2D(2, 2),

    layers.Conv2D(256, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(256, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.MaxPooling2D(2, 2),

    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.MaxPooling2D(2, 2),

    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(512, 3, padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.MaxPooling2D(2, 2),

    layers.Flatten(),

    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(num_classes, activation='sigmoid'),
])

In [ ]:
model = VGG
model.compile(optimizer='SGD',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy',])

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, monitor='loss')

In [60]:
epochs = 20

model_num = 1

In [ ]:
model_num += 1
history = model.fit(
  train_ds,
  epochs=epochs,
  callbacks=[es]
)

model.save(f'model_{model_num}.keras')

In [61]:
path_to_model = '/content/drive/MyDrive/VGG_19.keras'

In [62]:
model = tf.keras.models.load_model(path_to_model)

In [63]:
predictions = model.predict(test_ds)
y_pred = np.argmax(predictions, axis=1)

27/27 [==============================] - 509s 19s/step


In [87]:
val = model.predict(val_ds)
y_pred = np.argmax(val, axis=1)

2/2 [==============================] - 34s 13s/step


In [64]:
path_to_model_2 = '/content/drive/MyDrive/AlexNet.keras'

In [65]:
model_2 = tf.keras.models.load_model(path_to_model_2)

In [66]:
predictions_2 = model_2.predict(test_ds)
y_pred_2 = np.argmax(predictions_2, axis=1)

27/27 [==============================] - 18s 634ms/step


In [88]:
val_2 = model.predict(val_ds)
y_pred_2 = np.argmax(val_2, axis=1)

2/2 [==============================] - 39s 14s/step


In [67]:
path_to_model_3 = '/content/drive/MyDrive/custom.keras'

In [68]:
model_3 = tf.keras.models.load_model(path_to_model_3)

In [69]:
predictions_3 = model_3.predict(test_ds)
y_pred_3 = np.argmax(predictions_3, axis=1)

27/27 [==============================] - 16s 587ms/step


In [89]:
val_3 = model.predict(val_ds)
y_pred_3 = np.argmax(val_3, axis=1)

2/2 [==============================] - 33s 13s/step


In [70]:
y_true = np.concatenate([y for _, y in test_ds], axis=0)

In [91]:
y_true_val = np.concatenate([y for _, y in val_ds], axis=0)

In [71]:
print("VGG-19")
print('f1_score: ', evaluate_f1(y_true, y_pred))
print('Time: ', 19/32)

print('--------------------------')

print("Modified AlexNet")
print('f1_score: ', evaluate_f1(y_true, y_pred_2))
print('Time: ', 0.603/32)

print('--------------------------')

print("Custom")
print('f1_score: ', evaluate_f1(y_true, y_pred_3))
print('Time: ', 0.602/32)

VGG-19
f1_score:  0.9082349823336745
Time:  0.59375
--------------------------
Modified AlexNet
f1_score:  0.7937351398846648
Time:  0.01884375
--------------------------
Custom
f1_score:  0.8217842336687596
Time:  0.0188125


In [93]:
print("VGG-19")
print('f1_score: ', evaluate_f1(y_true_val, y_pred))

print('--------------------------')

print("Modified AlexNet")
print('f1_score: ', evaluate_f1(y_true_val, y_pred_2))

print('--------------------------')

print("Custom")
print('f1_score: ', evaluate_f1(y_true_val, y_pred_3))

VGG-19
f1_score:  0.29263936063936064
Time:  0.59375
--------------------------
Modified AlexNet
f1_score:  0.23951248751248752
Time:  0.01884375
--------------------------
Custom
f1_score:  0.14332467532467535
Time:  0.0188125
